In [12]:
from sympy import symbols, Eq, solve, Matrix
import sympy as sp

## Iterasi 1 titik

In [10]:
'''
1. Tentukan fungsi utk func_x, func_y, .. dari fungsi utama 
'''
x = symbols('x')
y = symbols('y')
n = 4  # Adjust the n

xys = [
    [3, -12]
]

func_x = (11 + 4*y - x) / (y)
func_y = (4 + x + 4*y) / (x)

for i in range(n):
    xi, yi = xys[-1]
    x_temp = func_x.subs({x: xi, y: yi}).evalf()
    y_temp = func_y.subs({x: xi, y: yi}).evalf()
    xys.append([x_temp, y_temp])
    
for iter, res in enumerate(xys):
    print(iter, res)

0 [3, -12]
1 [3.33333333333333, -13.6666666666667]
2 [3.43902439024390, -14.2000000000000]
3 [3.46753692889042, -14.3531914893617]
4 [3.47520639735821, -14.4036617497650]


## Newton

In [50]:
def find_first_jacobian(vars, equations, initial_guess):
    # variables = list(set().union(*[eq.free_symbols for eq in equations]))
    # jacobian = Matrix([[sp.diff(eq, var) for var in variables] for eq in equations])
    jacobian =[]
    
    for equation in equations:
        curr_calc = []
        for i, var in enumerate(vars):
            temp = sp.diff(equation, var)
            curr_calc.append(temp)
        jacobian.append(curr_calc)
        
    return jacobian

def find_first_approach(vars, xys, jacobian):
    js = []
    
    for iter, jacob in enumerate(jacobian):
        temp = []
        for j in jacob:
            dics = {}
            for sym, var in zip(vars, xys):
                dics[sym] = var
                print(dics)
            temp.append(j.subs(dics).evalf())
        js.append(temp)
    return js

In [54]:
x = symbols('x')
y = symbols('y')

func_x = (11 + 4*y - x) / (y)
func_y = (4 + x + 4*y) / (x)

xys = [
    [0, 0]
]

# print(sp.diff(func_x, y))

jacobian = find_first_jacobian([x, y], [func_x, func_y], xys)

# for iter, jacob in enumerate(jacobian):
#     for j in jacob:
#         curr = j
#         for i in range(2):
#             if i == 0:
#                 curr.subs({x: 1})
#             else:
#                 curr.subs({y: 1})
#         print(curr.evalf())
print(jacobian)
# print(find_first_approach([x, y], xys[0], jacobian))

[[-1/y, 4/y - (-x + 4*y + 11)/y**2], [1/x - (x + 4*y + 4)/x**2, 4/x]]
